<a href="https://colab.research.google.com/github/czephyr/AMD_project/blob/main/colab_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [3]:
!pip install -q findspark
!pip install -q pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.3 MB/s eta 0:00:00


In [4]:
os.environ['KAGGLE_USERNAME'] = "aaaaaaaaaaaaaaaaafff"
os.environ['KAGGLE_KEY'] = "75e26e145e3185803897967c59e5e8d2"
!kaggle datasets download -f amazon_reviews_us_Baby_v1_00.tsv cynthiarempel/amazon-us-customer-reviews-dataset
!kaggle datasets download -f amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv cynthiarempel/amazon-us-customer-reviews-dataset
!kaggle datasets download -f amazon_reviews_us_Digital_Video_Games_v1_00.tsv cynthiarempel/amazon-us-customer-reviews-dataset

 96% 331M/346M [00:02<00:00, 177MB/s]
100% 346M/346M [00:02<00:00, 132MB/s]
 96% 235M/245M [00:02<00:00, 98.6MB/s]
100% 245M/245M [00:02<00:00, 95.1MB/s]
 64% 17.0M/26.6M [00:00<00:00, 49.8MB/s]
100% 26.6M/26.6M [00:00<00:00, 61.5MB/s]


In [5]:
import zipfile
with zipfile.ZipFile("amazon_reviews_us_Baby_v1_00.tsv.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
with zipfile.ZipFile("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
with zipfile.ZipFile("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

### Data to adjacency list

In [6]:
# the dataset is composed by Baby items, Digital Music and Digital Videogames
# RDD (customer,product)
tsv0 = spark.read.option("header", "true").option("sep", "\t").csv("amazon_reviews_us_Baby_v1_00.tsv").select(["customer_id","product_parent"]).rdd
tsv1 = spark.read.option("header", "true").option("sep", "\t").csv("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv").select(["customer_id","product_parent"]).rdd
tsv2 = spark.read.option("header", "true").option("sep", "\t").csv("amazon_reviews_us_Digital_Video_Games_v1_00.tsv").select(["customer_id","product_parent"]).rdd
tsv = tsv0.union(tsv1).union(tsv2)
data = tsv
print(f"Rows in the dataset: {data.count()}")
data.take(1)

Rows in the dataset: 3587247


[Row(customer_id='9970739', product_parent='329991347')]

In [19]:
data.take(2)

[Row(customer_id='9970739', product_parent='329991347'),
 Row(customer_id='23538442', product_parent='646108902')]

In [20]:
# Extract id of Digital Videogames products in the database
video_games = tsv2.map(lambda x: x[1]).distinct()
video_games.take(2)

['603406193', '341969535']

In [21]:
# Extract id of Baby products in the database
baby = tsv0.map(lambda x: x[1]).distinct()
baby.take(2)

['634188771', '3470998']

In [22]:
# Extract id of Digital Music products in the database
music = tsv1.map(lambda x: x[1]).distinct()
music.take(2)

['262935067', '452318038']

In [23]:
#### CELL 5
# Create couples of items purchased by the same customer
# RDD [(customer1, (product1,product2)),(customer2, (product3,product4)),(...)]
data_joined = data.join(data)
data_joined.take(2)

[('37918955', ('854565914', '854565914')),
 ('37918955', ('854565914', '490481000'))]

In [24]:
#### CELL 6
# Remove duplicate item couples created by the join
# RDD [(customer1, (product1,product2)),(customer2, (product3,product4)),(...)]
data_filtered = data_joined.filter(lambda x : x[1][0]!= x[1][1])
data_filtered.take(2)

[('37918955', ('854565914', '490481000')),
 ('37918955', ('490481000', '854565914'))]

In [25]:
#### CELL 7
# Create list of links between products
# RDD [(product1,product2),(product3,product4),(...))]
edge_list = data_filtered.map(lambda x : x[1])
edge_list.take(2) 

[('854565914', '490481000'), ('490481000', '854565914')]

In [ ]:
#### CELL 8
# Make second item a list
# RDD [(node1, [node2]),(node1, [node2]),(...)]
adjacency1 = edge_list.mapValues(lambda v: [v])

# RDD [(node1,[node2,node3,...]),(...)]
adjacency_list = adjacency1.reduceByKey(lambda a,b: a + b)

adjacency_list.take(1)

### Calculate pagerank

In [ ]:
#### CELL 9
def calculate_initial_contribs(row):
    """
    This function takes elements from the joined dataset above and
    computes the contribution to each outgoing link based on the
    current rank.
    """
    item, array = row
    num_edges = len(array)
    for node in array:
        yield item, node, 1/num_edges

# transition matrix stored as triplets
# RDD [(product1,(product2,probability)),(product3,(product4,probability))]
trans_matrix = adjacency_list.flatMap(calculate_initial_contribs).cache()
trans_matrix.take(10) 

23/03/20 21:30:39 WARN BlockManager: Task 202 already completed, not releasing lock for rdd_70_0


[('439716423', '543525459', 0.0010204081632653062),
 ('439716423', '496365324', 0.0010204081632653062),
 ('439716423', '419902739', 0.0010204081632653062),
 ('439716423', '235995163', 0.0010204081632653062),
 ('439716423', '231583317', 0.0010204081632653062),
 ('439716423', '716235929', 0.0010204081632653062),
 ('439716423', '81415061', 0.0010204081632653062),
 ('439716423', '354873430', 0.0010204081632653062),
 ('439716423', '65780667', 0.0010204081632653062),
 ('439716423', '944771005', 0.0010204081632653062)]

In [ ]:
# Product names
keys = trans_matrix.map(lambda x : x[0]).distinct().collect()
# map product names to integers
indexmap = dict(zip(keys, range(len(keys))))

# map items to indexes
# RDD [(index_product1,(index_product2,probability)),(index_product3,(index_product4,probability))]
hashed_matrix = trans_matrix.map(lambda x: (indexmap[x[0]],(indexmap[x[1]],x[2])))
hashed_matrix.take(2)

23/03/20 21:30:54 WARN BlockManager: Task 347 already completed, not releasing lock for rdd_70_0


[(0, (569322, 0.0010204081632653062)), (0, (471457, 0.0010204081632653062))]

In [ ]:
import numpy as np

# Array of pagerank stored in memory
n = len(keys) # num of products in adjacency list
pgrnk = np.ones(n)/n # time t
previus_pgrnk = np.ones(n) # time -1
     
# MSE distance between previous pgrank iteration
def l2distance(v, q):   
    return sum([(q_el - v_el)**2 for v_el, q_el in zip(v, q)])

In [ ]:
#### CELL 15
tolerance = 10e-10
max_iterations = 300
i = 0
tax = 0.85

while(l2distance(previus_pgrnk, pgrnk) >= tolerance and i < max_iterations):
    previus_pgrnk = np.copy(pgrnk)

    page_rank_values = (hashed_matrix
                    .mapValues(lambda v: tax*(v[1]*pgrnk[v[0]]) + (1-tax)*(1/n)) 
                    .reduceByKey(lambda a, b: a+b)
                    .sortByKey()
                    .collect()
                    )
  
    pgrnk = np.array([c for (i, c) in page_rank_values])
    i=i+1

In [ ]:
# Avg pagerank
np.mean(pgrnk)

0.001135444161043827

### Topic aware page rank

In [ ]:
vgdict = video_games.collect()
vg_array = [1 if (x in vgdict) else 0 for x in indexmap]

In [ ]:
# Reset arrays
n = len(keys) # num of products in adjacency list
pgrnk = np.ones(n)/n # time t
previus_pgrnk = np.ones(n) # time -1


In [ ]:
#### CELL 18
tolerance = 10e-10
max_iterations = 300
i = 0
tax = 0.85

# size of the page in the topic set
len_topic = video_games.count()    

# if the page is in the selected topic set it's pgrank calculation will have
# the added weight from the teletrasportation
def calcPgRnk(row):
    return tax*(row[1]*pgrnk[row[0]]) + vg_array[row[0]]*(1-tax)*(1/len_topic)


while(l2distance(previus_pgrnk, pgrnk) >= tolerance or i < max_iterations):
  
    previus_pgrnk = np.copy(pgrnk) 
    page_rank_values = (hashed_matrix
                        .mapValues(calcPgRnk) 
                        .reduceByKey(lambda a, b: a+b) 
                        .sortByKey()
                        .collect()
                       )
  
    pgrnk = np.array([c for (i, c) in page_rank_values])
    i = i + 1

In [ ]:
# Avg pgrnk for videogame products
sum = 0
for pg,vg in zip(pgrnk,vg_array):
    sum = pg*vg + sum 

sum/len_topic

0.007904045707260303

In [ ]:
# Avg pgrnk for products in the other two categories
sum_others = 0
for pg,vg in zip(pgrnk,vg_array):
    sum_others = pg*int(not(vg)) + sum_others

sum_others / (len(pgrnk)-len_topic)

0.0005901374609417463